In [22]:
import requests
from bs4 import BeautifulSoup
import time

# ベースURL
base_url = 'https://www.musashino-u.ac.jp/'

# User-Agent設定（ブロック回避）
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# 結果を格納する辞書
visited = {}

# -------------------------
# 再帰関数の定義
# -------------------------
def crawl(url, depth=0, max_depth=1):
    """
    再帰的にリンクをたどってURLとタイトルを取得する関数
    depth: 現在の深さ
    max_depth: 最大の深さ（深すぎると無限ループになるので制限）
    """
    # すでに訪問済みならスキップ
    if url in visited:
        return
    
    # HTTPリクエスト送信
    try:
        res = requests.get(url, headers=headers, timeout=5)
        res.encoding = res.apparent_encoding
        if res.status_code != 200:
            print(f"⚠️ ステータスコード異常: {url} -> {res.status_code}")
            return
    except Exception as e:
        print(f"⚠️ エラー: {url} -> {e}")
        return

    # BeautifulSoupでHTML解析
    soup = BeautifulSoup(res.text, 'html.parser')

    # タイトルを取得（<title>タグから）
    title = soup.title.string.strip() if soup.title and soup.title.string else "（タイトルなし）"

    # 結果を保存
    visited[url] = title
    print(f"✅ {url} -> {title}")

    # 再帰の深さを制限
    if depth >= max_depth:
        return

    # ページ内のすべてのリンクを取得
    links = soup.find_all('a', href=True)

    for a in links:
        href = a['href']
        # 絶対URLに変換
        if href.startswith('/'):
            href = base_url.rstrip('/') + href
        elif href.startswith('#') or href.startswith('mailto:') or href.startswith('javascript:'):
            continue
        elif not href.startswith('http'):
            href = base_url.rstrip('/') + '/' + href

        # 武蔵野大学サイト内のリンクのみを再帰的に探索
        if href.startswith(base_url):
            time.sleep(0.5)  # サーバー負荷対策
            crawl(href, depth + 1, max_depth)

# -------------------------
# 実行部分
# -------------------------
crawl(base_url, max_depth=1)

print("\n=== 出力結果 ===")
print("{")
for k, v in visited.items():
    print(f"    '{k}': '{v}',")
print("}")


✅ https://www.musashino-u.ac.jp/ -> 武蔵野大学
✅ https://www.musashino-u.ac.jp/access.html -> 交通アクセス | 武蔵野大学
✅ https://www.musashino-u.ac.jp/admission/request.html -> 資料請求 | 入試情報 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/contact.html -> お問い合わせ | 武蔵野大学
✅ https://www.musashino-u.ac.jp/prospective-students.html -> 武蔵野大学で学びたい方 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/students.html -> 在学生の方 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/alumni.html -> 卒業生の方 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/parents.html -> 保護者の方 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/business.html -> 企業・研究者の方 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/guide/ -> 大学案内 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/guide/profile/ -> 大学紹介 | 大学案内 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/guide/activities/ -> 大学の取り組み | 大学案内 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/guide/campus/ -> キャンパス | 大学案内 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/guide/facility/ -> 附置機関・センター・附属施設 | 大学案内 | 武蔵野大学
✅ https://www.musashino-u.ac.jp/guide/information/ -> 情報公開 | 大学案内 | 武蔵